### StrOutputParser

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()


chat_model = ChatOpenAI(
    model="qwen-plus",
    api_key=os.getenv('DASHSCOPE_API_KEY'),
    base_url=os.getenv('DASHSCOPE_BASE_URL')
)

messages = [
    SystemMessage(content="将以下内容从英语翻译成中文"),
    HumanMessage(content="It's a nice day today"),
]

result = chat_model.invoke(messages)
print(type(result))
print(result)

parser = StrOutputParser()
#使用parser处理model返回的结果
response = parser.invoke(result)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='今天天气真好' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 31, 'total_tokens': 35, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-6c7bcff6-a77e-9b40-9059-9775f8c59457', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--1efd5131-89aa-49b2-89b1-0e8b49f4a081-0' usage_metadata={'input_tokens': 31, 'output_tokens': 4, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
<class 'str'>
今天天气真好


### JSON解析器 JsonOutputParser

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一个靠谱的{role}"),
    ("human","{question}")
])

parser = JsonOutputParser()

# 方式1：
# result = chat_model.invoke(chat_prompt_template.format_messages(role="人工智能专家",question="人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式"))
# print(result)
# print(type(result))
#
# parser.invoke(result)

# 方式2：
chain = chat_prompt_template | chat_model | parser
result = chain.invoke({"role":"人工智能专家","question" : "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式"})
print(result)
print(type(result))

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}
<class 'dict'>


### 自己指定JSON格式

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 手动定义JSON格式指令
format_instructions = """{
    "joke": "str  #完整的笑话内容",
    "explanation": "str  #笑话的文化背景解释"
}"""

# 提示模板
joke_query = "告诉我一个笑话。"

prompt_template = PromptTemplate(
    template="回答用户的查询。\n格式要求：{format_instructions}\n，问题是：{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

# 输出解析器
parser = JsonOutputParser()

chain = prompt_template | chat_model | parser
resp = chain.invoke({"query": joke_query})
print(resp)
print(type(resp))

{'joke': '有一天，一只鸭子走进一家药店，问：“你们有润唇膏吗？”药剂师说：“这里是药店，我们不卖润唇膏。”第二天，鸭子又来了，问：“你们有润唇膏吗？”药剂师生气地说：“我不是告诉过你了吗，我们不卖润唇膏！别再来了！”第三天，鸭子又来了，问：“你们有润唇膏吗？”药剂师怒吼道：“你要是再问一次，我就把你的嘴粘上！”鸭子眨眨眼，问：“那你们有胶水吗？”', 'explanation': '这是一个典型的西方幽默笑话，利用了重复和意外结局制造笑点。鸭子的执着和药剂师逐渐升级的反应形成反差，而鸭子在最后突然转向‘胶水’的提问，巧妙地呼应了药剂师的威胁，展现了美式幽默中常见的机智和夸张风格。这种笑话没有恶意，旨在通过逻辑的突然转折引发轻松的笑声。'}
<class 'dict'>


In [14]:
print(parser.get_format_instructions())

Return a JSON object.


###  日期解析器 DatetimeOutputParser

In [8]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z

Return ONLY this string, no other words!


In [9]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
# print(output_parser.get_format_instructions())

str_time = '1547-06-23T21:24:07.078384Z'
date_time = output_parser.parse(str_time)
print(type(str_time))
print(type(date_time))
print(date_time)

<class 'str'>
<class 'datetime.datetime'>
1547-06-23 21:24:07.078384


### 列表解析器 CommaSeparatedListOutputParser

In [10]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini" ,api_key=os.getenv('OPENAI_API_KEY'),base_url=os.getenv('OPENAI_BASE_URL'))
# 创建解析器
output_parser = CommaSeparatedListOutputParser()

# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
    "生成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={
    "format_instructions": output_parser.get_format_instructions()
    })

# 提示模板与输出解析器传递输出
# chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})
print(res)
print(type(res))

['热门电影', '经典影片', '动画电影', '科幻电影', '恐怖片']
<class 'list'>


### XML解析器 XMLOutputParser

In [13]:
from langchain_core.output_parsers import XMLOutputParser

output_parser = XMLOutputParser()
# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import XMLOutputParser


parser = XMLOutputParser()

# 提示模板
joke_query = "告诉我一个笑话。"

prompt_template = PromptTemplate(
    template="回答用户的查询。\n格式要求：{format_instructions}\n，问题是：{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_value = prompt_template.invoke({"query": joke_query})

res = chat_model.invoke(message)
print(res.content)

print(parser.invoke(res))

<joke>
   <content>为什么程序员喜欢暗处？因为他们不喜欢光线，光线会让他们的眼睛疲劳。</content>
</joke>
{'joke': [{'content': '为什么程序员喜欢暗处？因为他们不喜欢光线，光线会让他们的眼睛疲劳。'}]}
